In [1]:
import yaml
from pathlib import Path
from astropy.time import Time, TimeDelta

inputYAML = "/Users/karr/software/METIS_Simulations/ESO/recipes.yaml"
with Path(inputYAML).open(encoding="utf-8") as file:
        rcps=yaml.safe_load(file)
  
expandables = [
    "dit",
    "mjdobs",
]

from itertools import product
import numpy as np

tObs = Time("2024-01-02 00:45:10")
from astar_utils import NestedMapping
expandables = [
            "dit",
        ]

In [8]:
mjd = '2024-01-02T03:45:10'
from datetime import datetime
tObs = Time(datetime.strptime(mjd, '%Y-%m-%dT%H:%M:%S'))
print(tObs.mjd)

60311.15636574074


In [9]:
nCalib = 1
allParms = []

for name, recipe in rcps.items():
    
            # expand the expandables
    
            expanded = [key for key in expandables
                        if isinstance(recipe["properties"][key], list)]
            combos = product(*[recipe["properties"][key] for key in expanded])
    
            # get the mode and the prefix for the title; the latter is not needed for an
            # observation sequence
            
            mode = recipe["mode"]
            prefix = recipe["do.catg"]
            nObs = recipe["properties"]["nObs"]
    
            # cycle through the combos (may only be one)
            sequence = True
            for combo in combos:
    
                # extract the properties and combine with the combo dictionary
                
                combodict = dict(zip(expanded, combo))
                props = recipe["properties"] | combodict
    
                # if needed, keep a tally of the different sets of parameters that will be passed to simulate()
                if(nCalib > 0):
    
                    # a blank value of ndfilter_name if not explicitly given
                    try:
                        nfname = props["ndfilter_name"]
                    except:
                        nfname = ""
    
                    allParms.append((props['dit'],props["ndit"],props["catg"],props["tech"],props["type"],props["filter_name"],nfname))
    

In [15]:
        darks = []
        flats = []
        ilab = 0
        
        # first, figure out and tabulate what dit/ndit/filter/ndfilter are needed for the dataset
        
        for elem in set(allParms):
    
            # if DARK isn't in the TYPE, it needs a corresponding DARK
            if(np.all(["DARK" not in elem[4]])):
                darks.append((elem[0],elem[1],elem[3]))
    
                # if it's not a DARK, FLAT or DETLIN, it needs a flat image
                if(np.all(["FLAT" not in elem[4],"DETLIN" not in elem[4],"LMS" not in elem[3]])):
                    flats.append((elem[0],elem[1],elem[3],elem[5],elem[6]))
        sources = {}
        sources["sky"] = {'name': 'empty_sky', 'kwargs': {}}
        sources["lamp"] = {'name': 'flat_field', 'kwargs': {'temperature': 200, 'amplitude': 0, 'filter_curve': 'V', 'extend': 15}}
        # another list of tuples with input parameters
        # separate loops for teh darks, and twice for the flats (once for sky flats, once for lamp).
        # logically, we want to generate them sequentially by type
        calibSet = []
        for elem in set(darks):
            tDir = {}
            tDir['source'] = sources["sky"] 
            tDir['properties'] = {}
            tDir['properties']['dit'] = elem[0]
            tDir['properties']['ndit'] = elem[1]
            tDir['properties']['filter_name'] = "closed"
            tDir['properties']['catg'] = "CALIB"
            tDir['properties']['type'] = "DARK"
            tDir['properties']['tplname'] = "METIS_gen_cal_dark"
            tDir['properties']['nObs'] = nObs
            if(elem[7] != ""):
                rcps[label]['properties']['filter_name'] = elem[7]    
                    
            if(",LM" in elem[2]):
                tDir['do.catg'] = "DARK_LM_RAW"
                tDir['mode'] = "img_lm"
                tDir['properties']['tech'] = "IMAGE,LM"
            elif(",N" in elem[2]):        
                tDir['do.catg'] = "DARK_N_RAW"
                tDir['mode'] = "img_n"
                tDir['properties']['tech'] = "IMAGE,N"
            elif(np.any(["LMS" in elem[2],"IFU" in elem[2]])):
                tDir['do.catg'] = "DARK_IFU_RAW"
                tDir['mode'] = "lms"
                tDir['properties']['tech'] = "LMS"
                
            calibSet.append(tDir)
        aa = set(calibSet)
        for elem in calibSet:
            label = "A"+str(ilab).zfill(4)  # unique label for the recipe
            ilab += 1
            rcps[label] = tDir


        for elem in set(flats):
            tDir = {}
            tDir['source'] = sources["sky"] 
            tDir['properties'] = {}
            tDir['properties']['dit'] = elem[0]
            tDir['properties']['ndit'] = elem[1]
            tDir['properties']['filter_name'] = elem[3]
            tDir['properties']['catg'] = "CALIB"
            tDir['properties']['type'] = "FLAT,TWILIGHT"
            tDir['properties']['tplname'] = "METIS_img_lm_cal_TwilightFlat"
            tDir['properties']['nObs'] = nObs

            if(",LM" in elem[2]):
                tDir['do.catg'] = "DARK_LM_RAW"
                tDir['mode'] = "img_lm"
                tDir['properties']['tech'] = "IMAGE,LM"
            elif(",N" in elem[2]):        
                tDir['do.catg'] = "DARK_N_RAW"
                tDir['mode'] = "img_n"
                tDir['properties']['tech'] = "IMAGE,N"
            elif(np.any(["LMS" in elem[2],"IFU" in elem[2]])):
                tDir['do.catg'] = "DARK_IFU_RAW"
                tDir['mode'] = "lms"
                tDir['properties']['tech'] = "LMS"
            calibSet.append(tDir)
            
        aa = set(calibSet)
        for elem in calibSet:
            label = "A"+str(ilab).zfill(4)  # unique label for the recipe
            ilab += 1
            rcps[label] = tDir
        for elem in set(flats):
            tDir = {}
            tDir['source'] = sources["sky"] 
            tDir['properties'] = {}
            tDir['properties']['dit'] = elem[0]
            tDir['properties']['ndit'] = elem[1]
            tDir['properties']['filter_name'] = elem[3]
            tDir['properties']['catg'] = "CALIB"
            tDir['properties']['type'] = "FLAT,TWILIGHT"
            tDir['properties']['tplname'] = "METIS_img_lm_cal_InternalFlat"
            tDir['properties']['nObs'] = nObs

            if(",LM" in elem[2]):
                tDir['do.catg'] = "DARK_LM_RAW"
                tDir['mode'] = "img_lm"
                tDir['properties']['tech'] = "IMAGE,LM"
            elif(",N" in elem[2]):        
                tDir['do.catg'] = "DARK_N_RAW"
                tDir['mode'] = "img_n"
                tDir['properties']['tech'] = "IMAGE,N"
            elif(np.any(["LMS" in elem[2],"IFU" in elem[2]])):
                tDir['do.catg'] = "DARK_IFU_RAW"
                tDir['mode'] = "lms"
                tDir['properties']['tech'] = "LMS"
            calibSet.append(tDir)
            
        aa = set(calibSet)
        for elem in calibSet:
            label = "A"+str(ilab).zfill(4)  # unique label for the recipe
            ilab += 1
            rcps[label] = tDir

    
        # now we're going to call simulate for each of the unique combinations involved
        # by setting up the parameters dictionary and the filename in the same way as
        # is done in the main routine
    
        print(f"Running calibrations: {nObs} of each type")
        dProps={}
        aa = set(calibSet)
        ilab = 0
        for elem in set(calibSet):
            label = "A"+str(ilab).zfill(4)  # unique label for the recipe
            tDir = {}
            rcps[label]['do.catg'] = elem[3] 
            rcps[label]['mode'] = elem[5] 
            rcps[label]['source'] = sources[elem[8]] 
            rcps[label]['properties'] = {}
            rcps[label]['properties']['dit'] = elem[0]
            rcps[label]['properties']['ndit'] = elem[1]
            rcps[label]['properties']['filter_name'] = elem[6]
            rcps[label]['properties']['catg'] = "CALIB"
            rcps[label]['properties']['tech'] = elem[2]
            rcps[label]['properties']['type'] = elem[4]
            rcps[label]['properties']['tplname'] = elem[9]
            rcps[label]['properties']['nObs'] = nObs


            prefix = elem[3]
    
            # ndfilter if present
            if(elem[7] != ""):
                rcps[label]['properties']['filter_name'] = elem[7]    
 

Running calibrations: 1 of each type


In [25]:
def getFileName(sdate,)

[(10, 1, 'PUP,M'),
 (100, 3, 'RAVC,IFU'),
 (10, 1, 'IMAGE,LM'),
 (0.05, 1, 'LSS,LM'),
 (50.5, 1, 'LMS'),
 (0.25, 1, 'LSS,LM'),
 (10, 1, 'IMAGE,LM'),
 (1.0, 1, 'IMAGE,LM'),
 (0.25, 1, 'APP,LM'),
 (0.005, 1, 'LSS,N'),
 (0.25, 1, 'LSS,N'),
 (0.25, 1, 'IMAGE,N'),
 (10, 1, 'PUP,N'),
 (10, 1, 'LMS'),
 (1.0, 1, 'LMS'),
 (34.0, 1, 'IMAGE,LM'),
 (0.05, 1, 'LSS,LM'),
 (67.0, 1, 'IMAGE,N'),
 (10, 1, 'LMS'),
 (0.25, 1, 'IMAGE,LM'),
 (100, 1, 'IMAGE,N'),
 (100, 3, 'LMS'),
 (0.25, 1, 'LMS'),
 (34.0, 1, 'LMS'),
 (10, 1, 'IMAGE,N'),
 (0.25, 1, 'IMAGE,LM'),
 (0.25, 1, 'IMAGE,LM'),
 (0.05, 1, 'LSS,N'),
 (10, 1, 'IMAGE,LM'),
 (0.01, 1, 'IMAGE,N'),
 (0.25, 1, 'IMAGE,LM'),
 (100, 3, 'LMS'),
 (0.25, 1, 'LSS,LM'),
 (0.25, 1, 'APP,LM'),
 (10, 1, 'IMAGE,LM'),
 (100, 3, 'RAVC,IFU'),
 (0.25, 1, 'IMAGE,LM'),
 (83.5, 1, 'IMAGE,N'),
 (100, 1, 'LMS'),
 (17.5, 1, 'IMAGE,LM'),
 (100, 1, 'IMAGE,LM'),
 (0.25, 1, 'LMS'),
 (0.25, 4, 'IMAGE,LM'),
 (67.0, 1, 'IMAGE,LM'),
 (50.5, 1, 'IMAGE,LM'),
 (1.0, 1, 'IMAGE,N'),
 (34.0,

In [23]:
darks = []
flats = []
for elem in set(alls):
    if(np.all(["DARK" not in elem[4],"WAVE" not in elem[4]])):
        darks.append((elem[0],elem[1],elem[3]))
        if("FLAT" not in elem[4]):
            flats.append((elem[0],elem[1],elem[3],elem[5],elem[6]))

[(10, 1, 'PUP,M'),
 (100, 3, 'RAVC,IFU'),
 (10, 1, 'IMAGE,LM'),
 (0.05, 1, 'LSS,LM'),
 (50.5, 1, 'LMS'),
 (0.25, 1, 'LSS,LM'),
 (10, 1, 'IMAGE,LM'),
 (1.0, 1, 'IMAGE,LM'),
 (0.25, 1, 'APP,LM'),
 (0.005, 1, 'LSS,N'),
 (0.25, 1, 'LSS,N'),
 (0.25, 1, 'IMAGE,N'),
 (10, 1, 'PUP,N'),
 (10, 1, 'LMS'),
 (1.0, 1, 'LMS'),
 (34.0, 1, 'IMAGE,LM'),
 (0.05, 1, 'LSS,LM'),
 (67.0, 1, 'IMAGE,N'),
 (10, 1, 'LMS'),
 (0.25, 1, 'IMAGE,LM'),
 (100, 1, 'IMAGE,N'),
 (100, 3, 'LMS'),
 (0.25, 1, 'LMS'),
 (34.0, 1, 'LMS'),
 (10, 1, 'IMAGE,N'),
 (0.25, 1, 'IMAGE,LM'),
 (0.25, 1, 'IMAGE,LM'),
 (0.05, 1, 'LSS,N'),
 (10, 1, 'IMAGE,LM'),
 (0.01, 1, 'IMAGE,N'),
 (0.25, 1, 'IMAGE,LM'),
 (100, 3, 'LMS'),
 (0.25, 1, 'LSS,LM'),
 (0.25, 1, 'APP,LM'),
 (10, 1, 'IMAGE,LM'),
 (100, 3, 'RAVC,IFU'),
 (0.25, 1, 'IMAGE,LM'),
 (83.5, 1, 'IMAGE,N'),
 (100, 1, 'LMS'),
 (17.5, 1, 'IMAGE,LM'),
 (100, 1, 'IMAGE,LM'),
 (0.25, 1, 'LMS'),
 (0.25, 4, 'IMAGE,LM'),
 (67.0, 1, 'IMAGE,LM'),
 (50.5, 1, 'IMAGE,LM'),
 (1.0, 1, 'IMAGE,N'),
 (34.0,

In [6]:
def checkRecipes(rcps):

    """check recipe dictionary for necessary input"""

    ex1 = ["prefix","mode","properties"]
    ex2 = ["dit","ndit","filter_name","catg","tech","type","nObs"]

    for name, recipe in rcps.items():    
        for elem in ex1:
            if(elem not in recipe):
                print(f'Recipe {name} does not contain required field {elem}')
                return 1
        for elem in ex2:
            if(elem not in recipe["properties"]):
                print(f'Recipe {name} does not contain required field {elem}')
                return 1

    return 0
with Path(inputYAML).open(encoding="utf-8") as file:
        rcps=yaml.safe_load(file)
print(checkRecipes(rcps))

Recipe LM_LSS_SCI_RAW1 does not contain required field nObs
1


In [ ]:
ydict = {}
for elem in set(darksLM):
    